# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import numpy as np
import pandas as pd
import nltk
nltk.download(['punkt','wordnet'])
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, f1_score, make_scorer
import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sjhal\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sjhal\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql('DisasterResponse',engine)
X = df[['message']].message
Y = df[df.columns[4:]]

### 2. Write a tokenization function to process your text data

In [3]:
stop_words = stopwords.words("english")

def tokenize(text):
    words = word_tokenize(text.lower())
    lemmatizer = WordNetLemmatizer()
    
    tokens = [lemmatizer.lemmatize(word).lower().strip() for word in words if word not in stop_words]
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=10)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y)
pipeline.fit(X_train,Y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x000002C457D1ED30>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=10)))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
y_pred = pipeline.predict(X_test)

In [7]:
y_pred = pd.DataFrame(y_pred,columns=Y_test.columns)

In [8]:
for col in Y_test.columns:
    print(col)
    print(classification_report(Y_test[col], y_pred[col]))

related
              precision    recall  f1-score   support

         0.0       0.31      0.13      0.18      1533
         1.0       0.77      0.91      0.83      4965
         2.0       0.06      0.04      0.05        54

    accuracy                           0.72      6552
   macro avg       0.38      0.36      0.35      6552
weighted avg       0.65      0.72      0.67      6552

request
              precision    recall  f1-score   support

         0.0       0.84      0.97      0.90      5427
         1.0       0.38      0.08      0.13      1125

    accuracy                           0.82      6552
   macro avg       0.61      0.53      0.51      6552
weighted avg       0.76      0.82      0.77      6552

offer
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      6520
         1.0       0.00      0.00      0.00        32

    accuracy                           0.99      6552
   macro avg       0.50      0.50      0.50      655

C:\Users\sjhal\Anaconda3\envs\disaster\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sjhal\Anaconda3\envs\disaster\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sjhal\Anaconda3\envs\disaster\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sjha

              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99      6482
         1.0       0.00      0.00      0.00        70

    accuracy                           0.99      6552
   macro avg       0.49      0.50      0.50      6552
weighted avg       0.98      0.99      0.98      6552

refugees
              precision    recall  f1-score   support

         0.0       0.97      1.00      0.98      6325
         1.0       0.00      0.00      0.00       227

    accuracy                           0.96      6552
   macro avg       0.48      0.50      0.49      6552
weighted avg       0.93      0.96      0.95      6552

death
              precision    recall  f1-score   support

         0.0       0.96      1.00      0.98      6281
         1.0       0.06      0.00      0.01       271

    accuracy                           0.96      6552
   macro avg       0.51      0.50      0.49      6552
weighted avg       0.92      0.96      0.94      6552

othe

C:\Users\sjhal\Anaconda3\envs\disaster\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sjhal\Anaconda3\envs\disaster\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sjhal\Anaconda3\envs\disaster\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [9]:
pipeline.score(X_test, Y_test)

0.14713064713064714

In [10]:
np.mean(np.mean(Y_test.reset_index(drop=True).eq(y_pred), axis=0))

0.9226826414326413

In [11]:
df_add = Y_test.reset_index(drop=True).add(y_pred, fill_value=0)
(df_add[df_add == 2]/2).sum(axis=0, numeric_only=True).sum()/(Y_test.sum(axis=0, numeric_only=True).sum())

0.2725709372312984

### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x000002C457D1ED30>)),
  ('tfidf', TfidfTransformer()),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=10)))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x000002C457D1ED30>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=10)),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True

In [13]:
parameters = {'vect__max_df':[0.8, 0.9],
              'tfidf__smooth_idf':[True, False],
              'clf__estimator':[RandomForestClassifier(n_estimators=10),
                                RandomForestClassifier(n_estimators=20),
                                KNeighborsClassifier(n_neighbors=11),
                                KNeighborsClassifier(n_neighbors=17)]}

cv = GridSearchCV(estimator=pipeline, param_grid=parameters, n_jobs=11, cv=3, verbose=5)

In [14]:
cv.fit(X_train, Y_train)

Fitting 3 folds for each of 16 candidates, totalling 48 fits


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x000002C457D1ED30>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=10)))]),
             n_jobs=11,
             param_grid={'clf__estimator': [RandomForestClassifier(n_estimators=10),
                                            RandomForestClassifier(n_estimators=20),
                                            KNeighborsClassifier(n_neighbors=11),
                                            KNeighborsClassifier(n_neighbors=17)],
                         'tfidf__smooth_idf': [True, False],
                         'vect__max_df': [0.8, 0.9]},
             verbose=5)

In [15]:
print(" Results from Grid Search (model name: cv)" )
print("\n The best estimator across ALL searched params:\n",cv.best_estimator_)
print("\n The best score across ALL searched params:\n",cv.best_score_)
print("\n The best parameters across ALL searched params:\n",cv.best_params_)

 Results from Grid Search (model name: cv)

 The best estimator across ALL searched params:
 Pipeline(steps=[('vect',
                 CountVectorizer(max_df=0.8,
                                 tokenizer=<function tokenize at 0x000002C457D1ED30>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=KNeighborsClassifier(n_neighbors=17)))])

 The best score across ALL searched params:
 0.16148520853909343

 The best parameters across ALL searched params:
 {'clf__estimator': KNeighborsClassifier(n_neighbors=17), 'tfidf__smooth_idf': True, 'vect__max_df': 0.8}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [16]:
y_pred_cv = cv.predict(X_test)

In [17]:
y_pred_cv = pd.DataFrame(y_pred_cv,columns=Y_test.columns)

In [18]:
for col in Y_test.columns:
    print(col)
    print(classification_report(Y_test[col], y_pred_cv[col]))

related
              precision    recall  f1-score   support

         0.0       0.35      0.11      0.16      1533
         1.0       0.77      0.94      0.84      4965
         2.0       0.00      0.00      0.00        54

    accuracy                           0.74      6552
   macro avg       0.37      0.35      0.34      6552
weighted avg       0.66      0.74      0.68      6552

request
              precision    recall  f1-score   support

         0.0       0.83      0.98      0.90      5427
         1.0       0.39      0.07      0.11      1125

    accuracy                           0.82      6552
   macro avg       0.61      0.52      0.51      6552
weighted avg       0.76      0.82      0.77      6552

offer
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      6520
         1.0       0.00      0.00      0.00        32

    accuracy                           1.00      6552
   macro avg       0.50      0.50      0.50      655

C:\Users\sjhal\Anaconda3\envs\disaster\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sjhal\Anaconda3\envs\disaster\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sjhal\Anaconda3\envs\disaster\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sjha

              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99      6404
         1.0       0.00      0.00      0.00       148

    accuracy                           0.98      6552
   macro avg       0.49      0.50      0.49      6552
weighted avg       0.96      0.98      0.97      6552

missing_people
              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99      6482
         1.0       0.00      0.00      0.00        70

    accuracy                           0.99      6552
   macro avg       0.49      0.50      0.50      6552
weighted avg       0.98      0.99      0.98      6552

refugees
              precision    recall  f1-score   support

         0.0       0.97      1.00      0.98      6325
         1.0       0.00      0.00      0.00       227

    accuracy                           0.97      6552
   macro avg       0.48      0.50      0.49      6552
weighted avg       0.93      0.97      0.95      6

C:\Users\sjhal\Anaconda3\envs\disaster\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sjhal\Anaconda3\envs\disaster\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sjhal\Anaconda3\envs\disaster\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sjha

              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99      6474
         1.0       0.00      0.00      0.00        78

    accuracy                           0.99      6552
   macro avg       0.49      0.50      0.50      6552
weighted avg       0.98      0.99      0.98      6552

earthquake
              precision    recall  f1-score   support

         0.0       0.92      1.00      0.96      5926
         1.0       0.85      0.14      0.24       626

    accuracy                           0.92      6552
   macro avg       0.89      0.57      0.60      6552
weighted avg       0.91      0.92      0.89      6552

cold
              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99      6404
         1.0       0.00      0.00      0.00       148

    accuracy                           0.98      6552
   macro avg       0.49      0.50      0.49      6552
weighted avg       0.96      0.98      0.97      6552

oth

C:\Users\sjhal\Anaconda3\envs\disaster\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sjhal\Anaconda3\envs\disaster\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sjhal\Anaconda3\envs\disaster\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sjha

In [19]:
cv.score(X_test, Y_test)

0.1668192918192918

In [20]:
np.mean(np.mean(Y_test.reset_index(drop=True).eq(y_pred_cv), axis=0))

0.925273029439696

In [21]:
df_add = Y_test.reset_index(drop=True).add(y_pred_cv, fill_value=0)
(df_add[df_add == 2]/2).sum(axis=0, numeric_only=True).sum()/(Y_test.sum(axis=0, numeric_only=True).sum())

0.2751504729148753

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [22]:
# save the model to disk
filename = 'classifier.pkl'
pickle.dump(cv, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.